In [5]:
# import library
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd 
import numpy as np
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
print(selenium.__version__)


4.8.2


In [6]:
# set options
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])

# create driver
driver = webdriver.Chrome(options=chrome_options)

# go to musinsa codimap url
url = 'https://www.musinsa.com/app/codimap/lists'
driver.get(url)
try:
    driver.maximize_window()
except:
    pass

# go to most comment page
most_comment_button = driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/form/div[4]/div/div[1]/a[3]/span')
most_comment_button.click()

In [7]:
# crawling all page by scrolling until the bottom page

interval = 1

prev_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)") # scroll down
    time.sleep(interval) # give loading time
    curr_height = driver.execute_script("return document.body.scrollHeight") # store height

    if curr_height == prev_height:
        break # already read all page

    prev_height = curr_height # may can read more page


columns = ['codimap_category', 'codimap_title', 'codimap_date', 'views', 'comment_numbers']
codimaps = [] # store each codimap information respectively

soup = BeautifulSoup(driver.page_source, 'lxml') # crawling tool
data_rows = soup.find_all('li', attrs={'class':'style-list-item'})

for i, row in enumerate(data_rows):

    print(f"{i+1}번 째 코디맵 정보 크롤링")
    blank = [] # store i+1(th) codimap information

    # crawl category
    codimap_category = row.find('span', attrs={'class':'style-list-information__text'})
    if codimap_category:
        codimap_category = codimap_category.get_text().strip()
        blank.append(codimap_category)
    else:
        blank.append('NONE')
        print(f'{i+1}번 째 category error')
        continue
    
    # crawl title
    codimap_title = row.find('strong', attrs={'class':'style-list-information__title'})
    if codimap_title:
        codimap_title = codimap_title.get_text().strip()
        blank.append(codimap_title)
    else:
        blank.append('NONE')
        print(f'{i+1}번 째 title error')
        continue
    
    # crawl date
    codimap_date = row.find_all('span', attrs={'class':'post-information__text'})[0]
    if codimap_date:
        codimap_date = codimap_date.get_text().strip()
        blank.append(codimap_date)
    else:
        blank.append('NONE')
        print(f'{i+1}번 째 date error')
        continue

    # crawl views
    views = row.find_all('span', attrs={'class':'post-information__text'})[1]
    if views:
        views = views.get_text().strip()
        blank.append(views[2:])
    else:
        blank.append('NONE')
        print(f'{i+1}번 째 views error')
        continue

    # crawl comment numbers
    comment_numbers = row.find_all('span', attrs={'class':'post-information__text'})[2]
    if comment_numbers:
        comment_numbers = comment_numbers.get_text().strip()
        blank.append(comment_numbers[2:])
    else:
        blank.append('NONE')
        print(f'{i+1}번 째 comment numbers error')
        continue

    # add i+1(th) codimap information
    codimaps.append(blank)
    print('---------------------------------------------------')

# store all codimaps information
df = pd.DataFrame(codimaps, columns=columns)


1번 째 코디맵 정보 크롤링
---------------------------------------------------
2번 째 코디맵 정보 크롤링
---------------------------------------------------
3번 째 코디맵 정보 크롤링
---------------------------------------------------
4번 째 코디맵 정보 크롤링
---------------------------------------------------
5번 째 코디맵 정보 크롤링
---------------------------------------------------
6번 째 코디맵 정보 크롤링
---------------------------------------------------
7번 째 코디맵 정보 크롤링
---------------------------------------------------
8번 째 코디맵 정보 크롤링
---------------------------------------------------
9번 째 코디맵 정보 크롤링
---------------------------------------------------
10번 째 코디맵 정보 크롤링
---------------------------------------------------
11번 째 코디맵 정보 크롤링
---------------------------------------------------
12번 째 코디맵 정보 크롤링
---------------------------------------------------
13번 째 코디맵 정보 크롤링
---------------------------------------------------
14번 째 코디맵 정보 크롤링
---------------------------------------------------
15번 째 코디맵 정보 크롤링
--------------------------

In [11]:
# df.reset_index(drop=True)
display(df.head(10))
print(df.shape)

,codimap_category,codimap_title,codimap_date,views,comment_numbers
0,캐주얼,톤온톤 코디,21.07.02,"17,353",50
1,캐주얼,블랙 매니아,21.02.23,"36,385",47
2,캐주얼,오늘의 코디,22.03.22,"13,452",44
3,댄디,추천 댄디 룩,21.09.15,"21,647",32
4,캐주얼,릴랙스 룩,21.02.23,"24,073",25
5,스트릿,힙하다 힙해,21.02.03,"36,126",24
6,댄디,데이트 코디,21.12.24,"11,016",19
7,캐주얼,심플한 코디,21.07.30,"41,394",18
8,댄디,훈훈 남친 룩,21.03.24,"20,041",17
9,댄디,서머 데이트 룩,21.07.26,"8,390",17


(60, 5)


In [13]:
df.to_csv('무신사_코디맵_댓글순_크롤링.csv', encoding = 'utf-8-sig', index=False)